In [31]:
import pickle
import nltk
from transformers import pipeline, BartTokenizer, BartModel

In [32]:
dataset_file = open('eoc_df.pkl', 'rb')
eoc_df = pickle.load(dataset_file)
dataset_file.close()

In [3]:
summarizer = pipeline('summarization', model = 'facebook/bart-large-cnn')

In [20]:
example = eoc_df['text_cleaned'][0]
example

'evidence coverage 2023 unitedhealthcare dual complete hmopos dsnp tollfree 18664801086 tty 711 8 am8 pm local time 7 days week myuhcmedicarecom y0066eoch04320090002023c omb approval 09381051 expires february 29 2024 january 1 december 31 2023 evidence coverage medicare health benefits services prescription drug coverage member plan document gives details medicare health care prescription drug coverage january 1 december 31 2023 important legal document please keep safe place questions document please contact customer service 18664801086 tty users call 711 hours 8 am8 pm local time 7 days week plan unitedhealthcare dual complete hmopos dsnp insured unitedhealthcare insurance company one affiliates evidence coverage says us means unitedhealthcare says plan plan means unitedhealthcare dual complete hmopos dsnp unitedhealthcare discriminate basis race color national origin sex age disability health programs activities unitedhealthcare provides free services help communicate us documents l

In [7]:
print(summarizer(example, max_length=130, min_length=30, do_sample=False))

IndexError: index out of range in self

In [4]:
eoc_df['bart_summary'] = eoc_df.apply(lambda row : summarizer(row['text_cleaned'], max_length = 130, min_length = 30, do_sample = False), axis = 1)

Token indices sequence length is longer than the specified maximum sequence length for this model (56287 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

In [38]:
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartModel.from_pretrained("facebook/bart-large-cnn")

# generate chunks of text \ sentences <= 1024 tokens
# generate chunks of text \ sentences <= 1024 tokens
def nest_sentences(document):
  nested = []
  sent = []
  length = 0
  for sentence in nltk.sent_tokenize(document):
    length += len(sentence)
    if length < 1024:
      sent.append(sentence)
    else:
      nested.append(sent)
      sent = [sentence]
      length = len(sentence)

  if sent:
    nested.append(sent)
  return nested

# generate summary on text with <= 1024 tokens
def generate_summary(nested_sentences):
  device = 'cuda'
  summaries = []
  for nested in nested_sentences:
    input_tokenized = bart_tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = bart_model.to(device).generate(input_tokenized,
                                      length_penalty=3.0,
                                      min_length=30,
                                      max_length=100)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [39]:
generate_summary(example)

AssertionError: Torch not compiled with CUDA enabled